# Training Pipeline

quickstart: 一个完整的训练流程演示

# Stage 1: Continue Pretraining

第一阶段：PT(Continue PreTraining)增量预训练，在海量领域文本数据上二次预训练GPT模型，以适配领域数据分布

注意：
1. 此阶段是可选的，如果你没有海量领域文本，可以跳过此阶段，直接进行SFT阶段的有监督微调
2. 我实验发现：做领域知识注入，SFT比PT更高效，也可以跳过PT阶段

#### 说明：

以下 notebook/colab 代码为了快速验证训练代码可用，我们使用了小size的生成模型和小样本数据集，实际使用时，需要使用更大的模型和数据集，以获得更好的效果。

1. 生成模型：使用的是Bloom的`bigscience/bloomz-560m`
2. 数据集：PT阶段使用的是中文天龙八部小说部分文本和英文书籍部分文本，位于`data/pretrain`文件夹

## 配置运行环境


In [ ]:
!git clone --depth 1 https://github.com/shibing624/MedicalGPT.git
%cd MedicalGPT
%ls
!pip install -r requirements.txt

## Stage1 咱们开始吧

训练步骤如下：

1. 确认训练集
2. 执行训练脚本

训练脚本的执行逻辑如下：
1. 导入依赖包
2. 设置参数
3. 定义各函数并加载训练集
4. 加载模型和tokenizer
5. 开始训练并评估
6. 查看训练结果

**以下参数可以根据你的GPU实际情况修改，当前参数是根据Colab的T4单卡GPU（16GB显存）配置的**

In [1]:
%ls ./data/pretrain/

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
en_article_tail500.txt  fever.txt  tianlongbabu.txt


In [2]:
!python pretraining.py \
    --model_type bloom \
    --model_name_or_path bigscience/bloomz-560m \
    --train_file_dir ./data/pretrain \
    --validation_file_dir ./data/pretrain \
    --per_device_train_batch_size 3 \
    --per_device_eval_batch_size 3 \
    --do_train \
    --do_eval \
    --use_peft True \
    --seed 42 \
    --fp16 \
    --max_train_samples 20000 \
    --max_eval_samples 10 \
    --num_train_epochs 1 \
    --learning_rate 2e-4 \
    --warmup_ratio 0.05 \
    --weight_decay 0.01 \
    --logging_strategy steps \
    --logging_steps 10 \
    --eval_steps 50 \
    --evaluation_strategy steps \
    --save_steps 500 \
    --save_strategy steps \
    --save_total_limit 3 \
    --gradient_accumulation_steps 1 \
    --preprocessing_num_workers 1 \
    --block_size 128 \
    --group_by_length True \
    --output_dir outputs-pt-v1 \
    --overwrite_output_dir \
    --ddp_timeout 30000 \
    --logging_first_step True \
    --target_modules all \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0.05 \
    --torch_dtype float16 \
    --device_map auto \
    --report_to tensorboard \
    --ddp_find_unused_parameters False \
    --gradient_checkpointing True

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: python: command not found


这个脚本的重要参数可以分为几个类别，包括模型配置、训练控制、性能优化和日志记录。以下是每个类别的关键参数提炼：

模型配置
- `--model_type bloom`: 指定模型类型为Bloom。
- `--model_name_or_path bigscience/bloomz-560m`: 使用的预训练模型路径或名称。
- `--use_peft True`: 启用PEFT(Progressive Error Feedback Training)训练策略。
- `--lora_rank 8` 和 `--lora_alpha 16`: LoRA(Low-Rank Adaptation)的配置参数，用于调整模型的适应性。
- `--lora_dropout 0.05`: LoRA层的dropout率。

训练控制
- `--train_file_dir` 和 `--validation_file_dir`: 指定训练和验证数据集的目录。
- `--do_train` 和 `--do_eval`: 启用训练和评估。
- `--max_train_samples 20000` 和 `--max_eval_samples 10`: 分别限制训练和评估使用的最大样本数。
- `--num_train_epochs 1`: 训练的轮次。
- `--learning_rate 2e-4`: 学习率。
- `--warmup_ratio 0.05`: 预热比例。
- `--weight_decay 0.01`: 权重衰减，用于正则化。

性能优化
- `--fp16`: 启用半精度浮点数训练，减少内存使用并可能加速训练。
- `--gradient_accumulation_steps 1`: 梯度累积步骤，用于在内存限制下增加批量大小。
- `--preprocessing_num_workers 1`: 数据预处理的工作线程数。
- `--block_size 128`: 输入序列的最大长度。
- `--group_by_length True`: 根据序列长度对数据进行分组，以减少填充。

日志记录和保存
- `--logging_strategy steps` 和 `--logging_steps 10`: 日志记录策略和步骤。
- `--eval_steps 50`: 每隔多少步进行一次评估。
- `--save_steps 500`: 每隔多少步保存一次模型。
- `--output_dir outputs-pt-v1`: 输出目录，用于保存训练结果。
- `--overwrite_output_dir`: 如果输出目录已存在，覆盖它。

这些参数共同定义了训练的模型类型、数据、训练过程以及性能优化和日志记录的方式。

In [5]:
%ls -lh outputs-pt-v1

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
total 26M
-rw-r--r-- 1 root root 5.0K Jul  3 12:23 README.md
-rw-r--r-- 1 root root  699 Jul  3 12:23 adapter_config.json
-rw-r--r-- 1 root root  13M Jul  3 12:23 adapter_model.safetensors
-rw-r--r-- 1 root root  472 Jul  3 12:23 all_results.json
drwxr-xr-x 2 root root 4.0K Jul  3 12:22 checkpoint-500/
drwxr-xr-x 2 root root 4.0K Jul  3 12:23 checkpoint-822/
-rw-r--r-- 1 root root  263 Jul  3 12:23 eval_results.json
drwxr-xr-x 3 root root 4.0K Jul  3 12:20 runs/
-rw-r--r-- 1 root root  552 Jul  3 12:23 special_tokens_map.json
-rw-r--r-- 1 root root  14M Jul  3 12:23 tokenizer.json
-rw-r--r-- 1 root root 1004 Jul  3 12:23 tokenizer_config.json
-rw-r--r-- 1 root root  229 Jul  3 12:23 train_results.json
-rw-r--r-- 1 root root  19K Jul  3 12:23 trainer_state.json


In [6]:
%ls -lh outputs-pt-v1/runs

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
total 512
drwxr-xr-x 2 root root 4.0K Jul  3 12:23 Jul03_12-19-31_intern-studio-50002342/


模型训练结果：
- 使用lora训练模型，则保存的lora权重是`adapter_model.bin`, lora配置文件是`adapter_config.json`，合并到base model的方法见`merge_peft_adapter.py`
- 日志保存在`output_dir/runs`目录下，可以使用tensorboard查看，启动tensorboard方式如下：`tensorboard --logdir output_dir/runs --host 0.0.0.0 --port 8009`

TensorBoard 是 TensorFlow 的可视化工具，它允许用户查看训练过程中的各种指标，如损失函数、准确率等，以及模型的图结构和权重直方图。使用 TensorBoard 可以帮助开发者更直观地理解、调试和优化程序。

在您提到的脚本中，TensorBoard 被用来查看模型训练的日志。主要关注点包括：

1. **训练和验证损失**：观察模型在训练过程中损失函数的变化，帮助判断模型是否在学习和进步。
2. **准确率**：如果日志中包含准确率信息，可以观察模型在训练和验证数据集上的表现。
3. **学习率**：查看学习率随训练进程的变化，对于使用学习率调度器的训练非常有用。
4. **权重和偏置的分布**：TensorBoard 可以展示模型参数（权重和偏置）的分布和变化情况，有助于诊断模型训练过程中的问题，如梯度消失或爆炸。
5. **图结构**：查看模型的结构，确保模型的构建符合预期。

启动 TensorBoard 的命令如下所示，这个命令会启动一个 Web 服务器，您可以在浏览器中查看训练过程的可视化结果：

```bash
tensorboard --logdir ooutputs-pt-v1/runs --host 0.0.0.0 --port 8010
```

- `--logdir` 指定了日志文件的目录。
- `--host 0.0.0.0` 允许从任何 IP 地址访问 TensorBoard，增加了访问的灵活性。
- `--port 8009` 指定了访问 TensorBoard 的端口。

通过查看这些日志，开发者可以更好地理解模型的训练过程，及时发现并解决训练中的问题。

lora模型权重合并到base model，合并后的模型保存在`--output_dir`目录下，合并方法如下：

In [ ]:
!python merge_peft_adapter.py --model_type bloom \
    --base_model bigscience/bloomz-560m --lora_model outputs-pt-v1 --output_dir merged-pt/

In [ ]:
%ls -lh merged-pt/

In [ ]:
%cat merged-pt/config.json

Stage1 增量预训练完成。

# Stage 2: Supervised FineTuning

第二阶段：SFT(Supervised Fine-tuning)有监督微调，构造指令微调数据集，在预训练模型基础上做指令精调，以对齐指令意图，并注入领域知识

| Stage 2: Supervised Fine-tuning | [supervised_finetuning.py](https://github.com/shibing624/MedicalGPT/blob/main/supervised_finetuning.py) | [run_sft.sh](https://github.com/shibing624/MedicalGPT/blob/main/run_sft.sh)  |

#### 说明：
以下 notebook/colab 代码为了快速验证训练代码可用，我们使用了小size的生成模型和小样本数据集，实际使用时，需要使用更大的模型和数据集，以获得更好的效果。

1. 生成模型：使用的是Bloom的`bigscience/bloomz-560m` 或者 Stage1得到的预训练模型
2. 数据集：SFT阶段使用的是使用的是Belle的1千条抽样数据，位于`data/finetune`文件夹

## Stage2 咱们开始吧

训练步骤如下：

1. 确认训练集
2. 执行训练脚本

训练脚本的执行逻辑如下：
1. 导入依赖包
2. 设置参数
3. 定义各函数并加载训练集
4. 加载模型和tokenizer
5. 开始训练并评估
6. 查看训练结果

In [ ]:
%ls ./data/finetune

In [ ]:
!python supervised_finetuning.py \
    --model_type bloom \
    --model_name_or_path merged-pt \
    --train_file_dir ./data/finetune \
    --validation_file_dir ./data/finetune \
    --per_device_train_batch_size 4 \
    --per_device_eval_batch_size 4 \
    --do_train \
    --do_eval \
    --use_peft True \
    --fp16 \
    --max_train_samples 1000 \
    --max_eval_samples 10 \
    --num_train_epochs 1 \
    --learning_rate 2e-5 \
    --warmup_ratio 0.05 \
    --weight_decay 0.05 \
    --logging_strategy steps \
    --logging_steps 10 \
    --eval_steps 50 \
    --evaluation_strategy steps \
    --save_steps 500 \
    --save_strategy steps \
    --save_total_limit 3 \
    --gradient_accumulation_steps 1 \
    --preprocessing_num_workers 1 \
    --output_dir outputs-sft-v1 \
    --overwrite_output_dir \
    --ddp_timeout 30000 \
    --logging_first_step True \
    --target_modules all \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0.05 \
    --torch_dtype float16 \
    --device_map auto \
    --report_to tensorboard \
    --ddp_find_unused_parameters False \
    --gradient_checkpointing True

In [ ]:
%ls -lh outputs-sft-v1

模型训练结果：
- 使用lora训练模型，则保存的lora权重是`adapter_model.bin`, lora配置文件是`adapter_config.json`，合并到base model的方法见`merge_peft_adapter.py`
- 日志保存在`output_dir/runs`目录下，可以使用tensorboard查看，启动tensorboard方式如下：`tensorboard --logdir output_dir/runs --host 0.0.0.0 --port 8009`

lora模型权重合并到base model，合并后的模型保存在`--output_dir`目录下，合并方法如下：

In [ ]:
!python merge_peft_adapter.py --model_type bloom \
    --base_model merged-pt --lora_model outputs-sft-v1 --output_dir ./merged-sft

In [ ]:
%ls -lh merged-sft/

In [ ]:
%cat merged-sft/config.json

Stage2 SFT训练完成。

# Stage 3: DPO(Direct Preference Optimization)

第三阶段：DPO(Direct Preference Optimization)直接偏好优化，DPO通过直接优化语言模型来实现对其行为的精确控制，而无需使用复杂的强化学习，也可以有效学习到人类偏好，DPO相较于RLHF更容易实现且易于训练，效果更好

| Stage 3: Direct Preference Optimization        |  [dpo_training.py](https://github.com/shibing624/MedicalGPT/blob/main/dpo_training.py) | [run_dpo.sh](https://github.com/shibing624/MedicalGPT/blob/main/run_dpo.sh)    |

#### 说明：
以下 notebook/colab 代码为了快速验证训练代码可用，我们使用了小size的生成模型和小样本数据集，实际使用时，需要使用更大的模型和数据集，以获得更好的效果。

1. 生成模型：使用的是Bloom的`bigscience/bloomz-560m` 或者 Stage2得到的SFT模型
2. 数据集：DPO阶段使用的是医疗reward数据，抽样了500条，位于`data/reward`文件夹

## Stage3 咱们开始吧

训练步骤如下：

1. 确认训练集
2. 执行训练脚本

训练脚本的执行逻辑如下：
1. 导入依赖包
2. 设置参数
3. 定义各函数并加载训练集
4. 加载模型和tokenizer
5. 开始训练并评估
6. 查看训练结果

In [ ]:
%ls ./data/reward/

In [ ]:
!python dpo_training.py \
    --model_type bloom \
    --model_name_or_path ./merged-sft \
    --train_file_dir ./data/reward \
    --validation_file_dir ./data/reward \
    --per_device_train_batch_size 3 \
    --per_device_eval_batch_size 1 \
    --do_train \
    --do_eval \
    --use_peft True \
    --max_train_samples 1000 \
    --max_eval_samples 10 \
    --max_steps 100 \
    --eval_steps 10 \
    --save_steps 50 \
    --max_source_length 128 \
    --max_target_length 128 \
    --output_dir outputs-dpo-v1 \
    --target_modules all \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0.05 \
    --torch_dtype float16 \
    --fp16 True \
    --device_map auto \
    --report_to tensorboard \
    --remove_unused_columns False \
    --gradient_checkpointing True \
    --cache_dir ./cache

In [ ]:
%ls -lh outputs-dpo-v1

模型训练结果：
- 使用lora训练模型，则保存的lora权重是`adapter_model.bin`, lora配置文件是`adapter_config.json`，合并到base model的方法见`merge_peft_adapter.py`
- 日志保存在`output_dir/runs`目录下，可以使用tensorboard查看，启动tensorboard方式如下：`tensorboard --logdir output_dir/runs --host 0.0.0.0 --port 8009`

lora模型权重合并到base model，合并后的模型保存在`--output_dir`目录下，合并方法如下：

In [ ]:
!python merge_peft_adapter.py --model_type bloom \
    --base_model merged-sft --lora_model outputs-dpo-v1 --output_dir merged-dpo/

In [ ]:
%ls -lh merged-dpo/

In [ ]:
%cat merged-dpo/config.json

Stage3 偏好建模第一次训练完成。

**至此一个完整的训练流程演示完成。**

# Test

In [ ]:
!python inference.py --model_type bloom --base_model merged-dpo
# 或在shell中运行
# python inference.py --model_type bloom --base_model merged-dpo --interactive

Input:介绍下南京
Response:  南京市位于江苏省西南部，是全国首批历史文化名城、国家中心城市和自由贸易试验区。

完。
